# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)  
        # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow([
        'artist','firstName','gender',
        'itemInSession','lastName','length',
        'level','location','sessionId',
        'song','userId'
        ])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((
            row[0], row[2], row[3], row[4], 
            row[5], row[6], row[7], row[8], 
            row[12], row[13], row[16]
            ))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    # Connect to cluster
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [9]:
# Create a Keyspace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### SELECT artist, song, length from music_library WHERE sessionId = 338 AND itemInSession = 4

`Table Name: artist_library
column 1: sessionId
column 2: itemInSession
column 3: artist
column 4: song
column 5: length
PRIMARY KEY (sessionId, itemInSession)`

In [45]:
## Query 1: Give me the artist, song title and song's length in the music app history 
## that was heard during sessionId = 338, and itemInSession = 4
query = """
    CREATE TABLE IF NOT EXISTS artist_library (
        sessionId int, 
        itemInSession int, 
        artist text,
        song text, 
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [47]:
# Populate the artist_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [21]:
## Verify the data was entered into the table
query = "SELECT artist, song, length from music_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### SELECT artist, song, firstName, lastName WEHRE userId = 10 AND sessionId = 182

`Table Name: artist_user_library
column 1: userId
column 2: sessionId
column 3: itemInSession
column 4: artist
column 5: song
column 6: firstName
column 7: lastName
PRIMARY KEY (userId, sessionId, itemInSession)`

In [49]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession)
## and user (first and last name) for userid = 10, sessionid = 182
query = """
    CREATE TABLE IF NOT EXISTS artist_user_library (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )"""

try:
    session.execute(query)
except Exception as e:
    print(e)               

In [50]:
# Populate the artist_user_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_user_library "
        query = query + "(userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), \
                                line[0], line[9], line[1], line[4]))

In [51]:
## Verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName \
         FROM artist_user_library \
         WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### SELECT firstName, lastName from user_library WHERE song = 'All Hands Against His Own'

`Table Name: user_library
column 1: userId
column 2: firstName
column 3: lastName
column 4: song
PRIMARY KEY (song, userId)`

In [52]:
## Query 3: Give me every user name (first and last) in my music app history
## who listened to the song 'All Hands Against His Own'
query = """
    CREATE TABLE IF NOT EXISTS user_library (
        userId int,
        firstName text,
        lastName text,
        song text,
        PRIMARY KEY (song, userId)
    )"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [55]:
# Populate the user_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (userId, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [56]:
## Verify the data was entered into the table
query = "SELECT firstName, lastName from user_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [57]:
## Drop the table before closing out the sessions
query = "DROP TABLE artist_library"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_user_library"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [59]:
session.shutdown()
cluster.shutdown()